ДЗ
===
Использовать датасет MovieLens
Использовать любые модели
Добиться целевого значения метрики RMSE на тестовой выборке 0.87 и ниже.

http://surpriselib.com

In [17]:
import pandas as pd
import numpy as np
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
from surprise import SVD
from surprise import SVDpp
from surprise.model_selection import cross_validate

In [ ]:
pip install surprise

Использовать датасет на 1 млн записей не получилось, при обучении умирает ядро

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.drop(columns=['timestamp'], inplace=True)
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [5]:
df = pd.merge(movies, ratings, on='movieId', how='inner')
df.tail()

,movieId,title,genres,userId,rating
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5
100833,193585,Flint (2017),Drama,184,3.5
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5
100835,193609,Andrew Dice Clay: Dice Rules (1991),Comedy,331,4.0


In [6]:
['Ok' for x in df.isnull().any() if x == False ]

['Ok', 'Ok', 'Ok', 'Ok', 'Ok']

Создаем датасет для Surprise
========

In [7]:
dataset = pd.DataFrame({
    'uid': df.userId,
    'iid': df.title,
    'rating': df.rating
})
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [8]:
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

In [11]:
trainset, testset = train_test_split(data, test_size=0.15)

User-based
======

In [12]:
algo_user_true = KNNWithMeans(k=30 , sim_options={'name': 'pearson_baseline', 'user_based': True})
algo_user_true.fit(trainset)
test_pred_user_true = algo_user_true.test(testset)
acc = accuracy.rmse(test_pred_user_true, verbose=True)  

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8886


In [18]:
cross_validate(algo_user_true, data, measures=['RMSE', 'MAE'], cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9069  0.9003  0.9064  0.8942  0.8843  0.8984  0.0084  
MAE (testset)     0.6879  0.6805  0.6845  0.6775  0.6701  0.6801  0.0061  
Fit time          0.45    0.45    0.43    0.44    0.44    0.44    0.01    
Test time         1.48    1.45    1.41    1.41    1.39    1.43    0.03    


{'test_rmse': array([0.90692487, 0.90030729, 0.90636247, 0.8941788 , 0.88431887]),
 'test_mae': array([0.6878753 , 0.68051353, 0.68451338, 0.67752591, 0.6701101 ]),
 'fit_time': (0.4477813243865967,
  0.4464609622955322,
  0.4255990982055664,
  0.43622279167175293,
  0.4427978992462158),
 'test_time': (1.4762837886810303,
  1.4509057998657227,
  1.4065299034118652,
  1.4115707874298096,
  1.3852009773254395)}

item-based
========

In [19]:
algo_user_false = KNNWithMeans(k=60 , sim_options={'name': 'pearson_baseline', 'user_based': False})
algo_user_false.fit(trainset)
test_pred_user_false = algo_user_false.test(testset)
acc = accuracy.rmse(test_pred_user_false, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8752


In [20]:
cross_validate(algo_user_false, data, measures=['RMSE', 'MAE'], cv=5, verbose=True, n_jobs=-1)

/Users/andreysemeshin/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8760  0.8833  0.8802  0.8942  0.8864  0.8840  0.0061  
MAE (testset)     0.6650  0.6702  0.6699  0.6798  0.6699  0.6710  0.0048  
Fit time          13.52   14.88   14.13   13.30   11.06   13.38   1.28    
Test time         9.33    8.41    8.32    7.95    7.57    8.32    0.59    


{'test_rmse': array([0.87596501, 0.88333355, 0.88018714, 0.89415956, 0.88640006]),
 'test_mae': array([0.66498773, 0.67021138, 0.66985011, 0.67979722, 0.66994224]),
 'fit_time': (13.52240800857544,
  14.876564025878906,
  14.125750064849854,
  13.295965909957886,
  11.062235116958618),
 'test_time': (9.333152294158936,
  8.406702041625977,
  8.323076248168945,
  7.949490070343018,
  7.5742669105529785)}

SVD
======

In [21]:
algo_svd = SVD()
algo_svd.fit(trainset)
test_pred_svd = algo_svd.test(testset)
accuracy.rmse(test_pred_svd, verbose=True)

RMSE: 0.8692


0.8691700778368804

In [22]:
cross_validate(algo_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8771  0.8722  0.8717  0.8738  0.8787  0.8747  0.0028  
MAE (testset)     0.6761  0.6683  0.6713  0.6718  0.6740  0.6723  0.0026  
Fit time          4.83    4.97    4.88    4.86    4.75    4.86    0.07    
Test time         0.21    0.21    0.22    0.20    0.19    0.21    0.01    


{'test_rmse': array([0.87712211, 0.87223664, 0.87167165, 0.87383341, 0.87874566]),
 'test_mae': array([0.67607039, 0.66833248, 0.67126129, 0.67184402, 0.67401783]),
 'fit_time': (4.833492994308472,
  4.974617958068848,
  4.877691268920898,
  4.858327150344849,
  4.753335952758789),
 'test_time': (0.21120381355285645,
  0.2117137908935547,
  0.21555376052856445,
  0.20035624504089355,
  0.194443941116333)}

SVD++
=======

In [24]:
algo_svdpp = SVDpp()
algo_svdpp.fit(trainset)
test_pred_svdpp = algo_svdpp.test(testset)
accuracy.rmse(test_pred_svdpp, verbose=True)

RMSE: 0.8542


0.8541994247901679

In [25]:
cross_validate(algo_svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8616  0.8590  0.8532  0.8629  0.8731  0.8619  0.0065  
MAE (testset)     0.6566  0.6611  0.6554  0.6624  0.6670  0.6605  0.0042  
Fit time          563.41  557.99  562.16  563.01  536.01  556.51  10.43   
Test time         10.66   10.53   10.23   10.27   10.15   10.37   0.19    


{'test_rmse': array([0.86157098, 0.85896058, 0.85320076, 0.86285524, 0.87307421]),
 'test_mae': array([0.65660941, 0.66111866, 0.65544677, 0.66240402, 0.66702891]),
 'fit_time': (563.4069211483002,
  557.9871370792389,
  562.1606550216675,
  563.0061030387878,
  536.0055360794067),
 'test_time': (10.65798306465149,
  10.531361103057861,
  10.227113008499146,
  10.265458822250366,
  10.149238109588623)}

Итог
======

Метод SVD показал результат - 0.8691700778368804

Лучший результат дал метод SVD++ - 0.8541994247901679, однако время обучения модели составляет 15-20 минут